# Self-query with contextual compressor

* retriever:
  * `Self-querying retriever` --  use an LLM to construct new queries that can question the structured data/metadata of the document
  * `MultiQueryRetriever` -- allow an LLM to paraphrase the query to get hopefully a diverse set of docs
    * limited helpfulness for this application
  * `Contextual compression` -- use an LLM to pre-filter and compress the docs retrieved before feeding the contexts to another LLM to answer
    * ref: https://python.langchain.com/docs/modules/data_connection/retrievers/contextual_compression
* retrieval methods: cos/dot; llm-aided; MMR (Maximum marginal relevance)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from typing import Dict, List
from langchain_core.documents.base import Document
import langchain

In [3]:
import tomllib

with open('../.tokens.toml', 'rb') as f:
    _TOKENS = tomllib.load(f)

with open('../.config.toml', 'rb') as f:
    _CONFIGS = tomllib.load(f)

In [4]:
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_community.vectorstores import Redis

embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=_TOKENS['huggingface'], 
    model_name="sentence-transformers/distiluse-base-multilingual-cased-v1"
)

vs = Redis.from_existing_index(
    embedding=embeddings,
    index_name=_CONFIGS['index_name'],
    redis_url=_CONFIGS['redis_url'],
    schema=_CONFIGS['redis_schema'],
)

In [5]:
vs.similarity_search_with_score("谁是王尔德？")

Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']


[(Document(page_content='王尔德总共写过九部戏剧，但另外四部剧作已被遗忘。1895年2月14日，王尔德最后一部、也是他才华机智达到巅峰状态的剧作《认真的重要》在圣詹姆斯剧院首演，观众如潮，盛况空前。两周后，王尔德在阿尔比马尔俱乐部收到昆斯伯里侯爵约翰·道格拉斯留下的名片，上面写着：“致装模作样的好男色者奥斯卡·王尔德”。因为王尔德从1891年开始便与比他小十六岁的阿尔弗雷德·道格拉斯（侯爵之子，当时还在上牛津大学）有不正当关系，而且经常“俪影成双”地出现在伦敦的公共场所并一起旅游，侯爵十分反感。王尔德与年轻男性的同性恋行为遭人物议不自此时始。同年，王尔德以诽谤罪把昆斯伯里侯爵告上法庭，居然向他一贯通过自己的作品和生活方式加以百般嘲弄的维多利亚时代的英国法律和社会求助（两年后，王尔德沉痛地承认自己干了“一生中最可耻、最无法原谅、最可鄙的事 ”）。1895 年4月3日法院开庭审理王尔德诉昆斯伯里侯爵诽谤案（王尔德自幼渴望的倒是能在“女王诉王尔德”的官司中出庭），被告轻而易举地反证原告确系“好男色者”，从而推翻关于诽谤的指控。4月5日，侯爵无罪获释，王尔德反因涉嫌“同其他男子发生有伤风化的肉体关系”被捕后取保候审 。5月25日，伦敦中央刑事法院根据1885年通过的针对男子同性恋的刑法修正案判处王尔德两年劳役刑罚，先后囚于纽盖特、彭顿韦尔和万兹沃斯监狱，11月20日又移至雷丁监狱服满刑期，这位折翅的悖论大师从此一蹶不振。', metadata={'id': 'doc:notiondb:7aba74f2ecb54f879215fb8e88396210', 'author': '【英】奥斯卡·王尔德', 'name': '道连·葛雷的画像 【英】奥斯卡·王尔德', 'source': '读书笔记（文学）', 'tags': '小说', 'date_start': '20170920', 'date_end': None}),
  0.6278),
 (Document(page_content='【作者简介】\n奥斯卡·王尔德（Oscar Wilde，1854年10月16日—1900年11月30日），出生于爱尔兰都柏林，19世纪英国（准确来讲是爱尔兰，但是当时由英国统治）最伟大的作家与艺术家之一，以其剧作、诗歌、童话和小说闻名，唯美主义代表人物，19世纪

In [6]:
# from langchain.vectorstores.redis import RedisFilter, RedisNum, RedisText

# f = RedisText("author") % "*冯友兰*"
# vs.similarity_search_with_score('谁说过陌生贵己？', filter=f, k=3)

In [7]:
metadata_set = set([x['name'] for x in vs.schema['text']] + [x['name'] for x in vs.schema['numeric']])

In [8]:
metadata = _CONFIGS['attributes']
metadata

{'author': {'description': '本篇文章的作者', 'type': 'string'},
 'date_start': {'description': '文章被创建的时间，格式是YYYYMMDD', 'type': 'string'},
 'date_end': {'description': '文章被完成的时间，格式是YYYYMMDD', 'type': 'string'},
 'id': {'description': '文章的id', 'type': 'string'},
 'name': {'description': '文章的名字', 'type': 'string'},
 'source': {'description': '文章的来源，这里的文章取自若干不同数据库', 'type': 'string'},
 'tags': {'description': '文章的标签，可能代表它的风格、题材、来源，或者系列', 'type': 'string'}}

In [9]:
# ensure there's no more undocumented metadata
assert metadata_set.union(metadata.keys()) == metadata_set

In [10]:
metadata_set

{'author', 'content', 'date_end', 'date_start', 'id', 'name', 'source', 'tags'}

In [11]:
from langchain_community.llms import LlamaCpp


llm = LlamaCpp(
    model_path=_CONFIGS['model_path']+'/'+'Qwen-7B-Chat.Q4_K_M.gguf',
    name='Qwen/Qwen-7B-Chat', 
    **_CONFIGS['llm']
)

# llm = LlamaCpp(
#     model_path=_CONFIGS['model_path']+'/'+'qwen1_5-7b-chat-q4_0.gguf',
#     name='Qwen/Qwen1.5-7B-Chat', 
#     **_CONFIGS['llm']
# )

# llm = LlamaCpp(
#     model_path=_CONFIGS['model_path']+'/'+'zephyr-7b-beta.Q4_K_M.gguf',
#     name='HuggingFaceH4/zephyr-7b-beta', 
#     **_CONFIGS['llm']
# )

# llm = LlamaCpp(
#     model_path=_CONFIGS['model_path']+'/'+'chinese-alpaca-2-7b-q4_0.gguf',
#     name='hfl/chinese-alpaca-2-7b', 
#     **_CONFIGS['llm']
# )

/Users/fred/micromamba/envs/my-notion-companion/lib/python3.12/site-packages/langchain_core/utils/utils.py:159: UserWarning: WARNING! conversation is not default parameter.
                conversation was transferred to model_kwargs.
                Please confirm that conversation is what you intended.
  warnings.warn(
llama_model_loader: loaded meta data with 19 key-value pairs and 259 tensors from /Users/fred/Documents/models/Qwen-7B-Chat.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen
llama_model_loader: - kv   1:                               general.name str              = Qwen
llama_model_loader: - kv   2:                        qwen.context_length u32              = 32768
llama_model_loader: - kv   3:                           qwen.block_count u32              = 32
llama_model_loader: - kv   4:

In [12]:
from langchain.chains.query_constructor.base import AttributeInfo

attribute_info = list()

for k, v in metadata.items():
    attribute_info.append(
        AttributeInfo(
            name=k,
            description=v['description'],
            type=v['type']
        )
    )

attribute_info

[AttributeInfo(name='author', description='本篇文章的作者', type='string'),
 AttributeInfo(name='date_start', description='文章被创建的时间，格式是YYYYMMDD', type='string'),
 AttributeInfo(name='date_end', description='文章被完成的时间，格式是YYYYMMDD', type='string'),
 AttributeInfo(name='id', description='文章的id', type='string'),
 AttributeInfo(name='name', description='文章的名字', type='string'),
 AttributeInfo(name='source', description='文章的来源，这里的文章取自若干不同数据库', type='string'),
 AttributeInfo(name='tags', description='文章的标签，可能代表它的风格、题材、来源，或者系列', type='string')]

# Construct customized self-query retriever

Q: Why not using the standard?
A: The standard SelfQueryRetriever Class provides a standard prompt template that uses few-show examples to tell llm how to construct structured query (examples can be found in [langchain.chains.query_constructor.prompt](https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/chains/query_constructor/prompt.py). Most examples inside uses EQ (=) comparator, which isn't suitable for our use cases (mostly fuzzy matches). Therefore, we will reconstruct the self-query retriever using a customized few-shot prompt teamplate.

Q: why do we copied the `get_query_constructor_prompt` provided?
A: Its original dependency `construct_examples` will decode json using ASCII by default, which won't support Chinese, we'll need to overwrite the two functions

```python
retriever = SelfQueryRetriever.from_llm(
    llm=llm,
    vectorstore=vs_chroma,
    document_contents='Articles and excerpts.',
    metadata_field_info=metadata_field_info,
)
```

References: 
https://python.langchain.com/docs/modules/data_connection/retrievers/self_query/#constructing-from-scratch-with-lcel

In [13]:
from pprint import pprint

with open('../self_query_examples.toml', 'rb') as f:
    self_query_examples = tomllib.load(f)

pprint(self_query_examples['example'][0])

{'structured_request': {'filter': 'or(like("source", "小说"), like("tags", '
                                  '"小说"), like("name", "小说"))',
                        'query': '谁是王尔德？'},
 'user_query': '谁是王尔德？仅从“小说”中找答案。'}


In [14]:
# with open('../self_query_template_chinese.txt', 'r') as f:
#     self_query_template = "\n".join(f.readlines())

with open('../self_query_template.txt', 'r') as f:
    self_query_template = "\n".join(f.readlines())

In [15]:
from typing import Sequence, Union, Tuple
import json
from langchain.chains.query_constructor.base import _format_attribute_info
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain.chains.query_constructor.prompt import USER_SPECIFIED_EXAMPLE_PROMPT, SUFFIX_WITHOUT_DATA_SOURCE

def _format_attribute_info(info: Sequence[Union[AttributeInfo, dict]]) -> str:
    info_dicts = {}
    for i in info:
        i_dict = dict(i)
        info_dicts[i_dict.pop("name")] = i_dict
    # return json.dumps(info_dicts, indent=4, ensure_ascii=False).replace("{", "{{").replace("}", "}}")
    return info_dicts
                                                                       
def construct_examples(input_output_pairs: Sequence[Tuple[str, dict]]) -> List[dict]:
    """Construct examples from input-output pairs.

    Adapted from: https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/chains/query_constructor/base.py
    """
    examples = []
    for i, (_input, output) in enumerate(input_output_pairs):
        structured_request = (
            json.dumps(output, indent=4, ensure_ascii=False).replace("{", "{{").replace("}", "}}")
        )
        example = {
            "i": i + 1,
            "user_query": _input,
            "structured_request": structured_request,
        }
        examples.append(example)
    return examples

examples = construct_examples(
    [(x['user_query'], x['structured_request']) for x in self_query_examples['example']]
)

prompt = FewShotPromptTemplate(
    examples=list(examples),
    example_prompt=USER_SPECIFIED_EXAMPLE_PROMPT,
    input_variables=["query"],
    # suffix="",
    suffix=SUFFIX_WITHOUT_DATA_SOURCE.format(i=len(examples) + 1),
    prefix=self_query_template.format(
        content_and_attributes=json.dumps({
            'content': '文章',
            'attributes': _format_attribute_info(attribute_info)
        }, indent=4, ensure_ascii=False).replace("{", "{{").replace("}", "}}"),
        attributes_set=str(list(metadata_set))
    )
)

In [16]:
prompt.pretty_print()

Your goal is to structure the user's query to match the request schema provided below.



<< Structured Request Schema >>

When responding use a markdown code snippet with a JSON object formatted in the following schema:



```json

{

    "query": string \ text string to compare to document contents

    "filter": string \ logical condition statement for filtering documents

}

```



The query string should contain only text that is expected to match the contents of documents. Any conditions in the filter should not be mentioned in the query as well.



Make sure that you only use the comparators and logical operators listed above and no others.

Make sure that filters only use the attributed names with its function names if there are functions applied on them.

Make sure that filters only use format `YYYYMMDD` when handling date data typed values.

Make sure that filters take into account the descriptions of attributes and only make comparisons that are feasible given the type of da

In [17]:
from langchain.chains.query_constructor.base import StructuredQueryOutputParser
output_parser = StructuredQueryOutputParser.from_components()

In [18]:
%%time

langchain.debug = False

query_constructor = prompt | llm | output_parser
query_constructor.invoke({
    "query": "“每个人都以为他自己至少有一种主要的美德。”是出自哪里？请从“读书笔记（文学）”中找到答案。"
})


llama_print_timings:        load time =   12224.00 ms
llama_print_timings:      sample time =      91.65 ms /   256 runs   (    0.36 ms per token,  2793.33 tokens per second)
llama_print_timings: prompt eval time =   12222.24 ms /  1156 tokens (   10.57 ms per token,    94.58 tokens per second)
llama_print_timings:        eval time =   13414.72 ms /   255 runs   (   52.61 ms per token,    19.01 tokens per second)
llama_print_timings:       total time =   27395.62 ms /  1411 tokens


CPU times: user 14.4 s, sys: 2.39 s, total: 16.8 s
Wall time: 27.5 s


StructuredQuery(query='每个人都以为他自己至少有一种主要的美德。', filter=Operation(operator=<Operator.OR: 'or'>, arguments=[Comparison(comparator=<Comparator.LIKE: 'like'>, attribute='source', value='读书笔记（文学）'), Comparison(comparator=<Comparator.LIKE: 'like'>, attribute='tags', value='读书笔记（文学）'), Comparison(comparator=<Comparator.LIKE: 'like'>, attribute='name', value='每个人都以为他自己至少有一种主要的美德'), Comparison(comparator=<Comparator.LIKE: 'like'>, attribute='author', value='每个人都以为他自己至少有一种主要的美德')]), limit=None)

In [19]:
from langchain.retrievers.self_query.redis import RedisTranslator
from langchain.retrievers import SelfQueryRetriever

retriever = SelfQueryRetriever(
    query_constructor=prompt | llm | output_parser,
    vectorstore=vs,
    # requires a RedisModel object as input
    # can use the private ._schema attribute
    # ref: https://github.com/langchain-ai/langchain/blob/d7c26c89b2d4f5ff676ba7c3ad4f9075d50a8ab7/libs/community/langchain_community/vectorstores/redis/base.py#L572
    structured_query_translator=RedisTranslator(schema=vs._schema),
    enable_limit=True,
)

In [20]:
langchain.debug = False

q = '人生有几个不捡？仅从“笑死”中找答案。'

new_query, search_kwargs = retriever._prepare_query(
    q,
    structured_query=query_constructor.invoke({'query': q}),
)

print(new_query, str(search_kwargs['filter']))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   12224.00 ms
llama_print_timings:      sample time =      90.43 ms /   256 runs   (    0.35 ms per token,  2830.83 tokens per second)
llama_print_timings: prompt eval time =     263.70 ms /    19 tokens (   13.88 ms per token,    72.05 tokens per second)
llama_print_timings:        eval time =   11319.73 ms /   255 runs   (   44.39 ms per token,    22.53 tokens per second)
llama_print_timings:       total time =   13296.54 ms /   274 tokens


人生有几个不捡？ (@source:(笑死) | (@tags:(笑死) | @name:(笑死)))


In [21]:
%%time 

langchain.debug = False
retriever.invoke('道连是哪本小说中出现的人物？')

Llama.generate: prefix-match hit

llama_print_timings:        load time =    9236.08 ms
llama_print_timings:      sample time =       9.06 ms /    54 runs   (    0.17 ms per token,  5959.61 tokens per second)
llama_print_timings: prompt eval time =     299.90 ms /    16 tokens (   18.74 ms per token,    53.35 tokens per second)
llama_print_timings:        eval time =    1776.64 ms /    53 runs   (   33.52 ms per token,    29.83 tokens per second)
llama_print_timings:       total time =    2210.99 ms /    69 tokens
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name',

CPU times: user 882 ms, sys: 61.5 ms, total: 943 ms
Wall time: 2.46 s


[Document(page_content='“道连，你是生活中的任性而快活的人，像你这样的人不会去犯罪。不过你对我的恭维还是使我感到荣幸。现在请你告诉我——请把火柴递给我，好孩子，谢谢！——你跟西碧儿·韦恩现在究竟是怎样的关系？”\n道连·格雷霍地跳了起来，两腮通红，双目怒睁。“亨利！西碧儿·韦恩是神圣的！”\n“\n只有神圣的东西才值得去碰\n，道连，”亨利勋爵的声调出人意料地稍带几分激昂。“你何必发火呢？我料想她总有一天会属于你的。\n恋爱中的人总是先欺骗自己，最后欺骗别人。这就是大家所说的罗曼司。\n”\n\n\n“哦，贝泽尔是个再好不过的人，可我觉得他有那么一点儿迂腐。亨利，自从我和你认识以后，我有这样的感觉。”\n“老弟，贝泽尔把他身上的全部可爱的气质都放到创作中去了。结果他为生活留下的就只有他的偏见、准则和大道理。我所认识的艺术家中讨人喜欢的都是不成器的。有才气的艺术家只存在于他们的创作中，而他们本人都是索然无味的。\n一个伟大的诗人，一个真正伟大的诗人，是最没有诗意的人。但是等而下之的诗人却极其讨人喜欢。他们的诗写得愈糟糕，他们的外貌就愈生动。如果一个诗人出版了一本二三流的十四行诗集，此人一定具有不可抗拒的魅力。他把写不出来的诗都在生活中实现了。而另一类诗人却把他们不敢身体力行的意境都写成了诗。\n”\n……（略）\n诚然，很少有人像道连·格雷那样吸引他，但这少年疯狂地热恋着另外一个人，却丝毫没有引起他的不快或妒意。他反而感到高兴。这为他提供了一个更加饶有兴味的研究课题。亨利勋爵一向醉心于自然科学的方法，但是自然科学的一般研究对象在他看来却是乏味的，不足道的。于是他始而开始解剖自己，继而解剖别人。在他心目中唯一值得加以研究的就是人生。与此相比，任何别的东西都毫无价值。确实，你要观察人生在痛苦和欢乐的奇特熔炉中的冶炼过程，不能戴上玻璃面罩，也免不了被硫磺味熏昏头脑，弄得想象中尽是牛鬼蛇神、噩梦凶兆。有些毒物是很难捉摸的，你要了解它们的特性，非得先中毒不可。然而，你得到的回报将是不可估量的！整个世界在你心目中将变得无比奇妙！探明\n高度严谨的情欲逻辑和涂上感情色彩的理性生活\n，观察它们何处相遇，何处分离，在哪一点上协调，在哪一点上不谐——真是其乐无穷！……（略）', metadata={'id': 'doc:notiondb:be9cfbb990cd

In [22]:
%%time 

langchain.debug = False
retriever.invoke('谁是王尔德？仅从“小说”中找答案。')

Llama.generate: prefix-match hit

llama_print_timings:        load time =    7427.01 ms
llama_print_timings:      sample time =      87.08 ms /   256 runs   (    0.34 ms per token,  2939.99 tokens per second)
llama_print_timings: prompt eval time =     250.05 ms /    19 tokens (   13.16 ms per token,    75.99 tokens per second)
llama_print_timings:        eval time =   11228.48 ms /   255 runs   (   44.03 ms per token,    22.71 tokens per second)
llama_print_timings:       total time =   13113.11 ms /   274 tokens
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name',

CPU times: user 9.27 s, sys: 266 ms, total: 9.53 s
Wall time: 13.4 s


[Document(page_content='王尔德总共写过九部戏剧，但另外四部剧作已被遗忘。1895年2月14日，王尔德最后一部、也是他才华机智达到巅峰状态的剧作《认真的重要》在圣詹姆斯剧院首演，观众如潮，盛况空前。两周后，王尔德在阿尔比马尔俱乐部收到昆斯伯里侯爵约翰·道格拉斯留下的名片，上面写着：“致装模作样的好男色者奥斯卡·王尔德”。因为王尔德从1891年开始便与比他小十六岁的阿尔弗雷德·道格拉斯（侯爵之子，当时还在上牛津大学）有不正当关系，而且经常“俪影成双”地出现在伦敦的公共场所并一起旅游，侯爵十分反感。王尔德与年轻男性的同性恋行为遭人物议不自此时始。同年，王尔德以诽谤罪把昆斯伯里侯爵告上法庭，居然向他一贯通过自己的作品和生活方式加以百般嘲弄的维多利亚时代的英国法律和社会求助（两年后，王尔德沉痛地承认自己干了“一生中最可耻、最无法原谅、最可鄙的事 ”）。1895 年4月3日法院开庭审理王尔德诉昆斯伯里侯爵诽谤案（王尔德自幼渴望的倒是能在“女王诉王尔德”的官司中出庭），被告轻而易举地反证原告确系“好男色者”，从而推翻关于诽谤的指控。4月5日，侯爵无罪获释，王尔德反因涉嫌“同其他男子发生有伤风化的肉体关系”被捕后取保候审 。5月25日，伦敦中央刑事法院根据1885年通过的针对男子同性恋的刑法修正案判处王尔德两年劳役刑罚，先后囚于纽盖特、彭顿韦尔和万兹沃斯监狱，11月20日又移至雷丁监狱服满刑期，这位折翅的悖论大师从此一蹶不振。', metadata={'id': 'doc:notiondb:7aba74f2ecb54f879215fb8e88396210', 'author': '【英】奥斯卡·王尔德', 'name': '道连·葛雷的画像 【英】奥斯卡·王尔德', 'source': '读书笔记（文学）', 'tags': '小说', 'date_start': '20170920', 'date_end': None}),
 Document(page_content='【作者简介】\n奥斯卡·王尔德（Oscar Wilde，1854年10月16日—1900年11月30日），出生于爱尔兰都柏林，19世纪英国（准确来讲是爱尔兰，但是当时由英国统治）最伟大的作家与艺术家之一，以其剧作、诗歌、童话和小说闻名，唯美主义代表人物，19世纪80年代美学运动的主力和9

In [23]:
%%time 

langchain.debug = False
retriever.invoke('人生有几个不捡？仅从“笑死”中找答案。')

Llama.generate: prefix-match hit

llama_print_timings:        load time =    7427.01 ms
llama_print_timings:      sample time =      87.61 ms /   256 runs   (    0.34 ms per token,  2922.07 tokens per second)
llama_print_timings: prompt eval time =     268.04 ms /    19 tokens (   14.11 ms per token,    70.88 tokens per second)
llama_print_timings:        eval time =   11229.68 ms /   255 runs   (   44.04 ms per token,    22.71 tokens per second)
llama_print_timings:       total time =   13145.71 ms /   274 tokens
Metadata key author not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key tags not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key date_start not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'so

CPU times: user 9.28 s, sys: 263 ms, total: 9.55 s
Wall time: 13.2 s


[Document(page_content='人生四不捡：塔吊下边冰红茶，铁轨上边牛肉干，过山车下八宝粥，课桌后边葡萄干', metadata={'id': 'doc:notiondb:d6d5faf38d8e4b1d9d9a6a19c00bc9dd', 'author': None, 'name': '笑死', 'source': '格言小集', 'tags': None, 'date_start': None, 'date_end': None})]

In [24]:
%%time 

langchain.debug = True
retriever.invoke('请列举NBA著名球星3名')

Llama.generate: prefix-match hit


[chain/start] [1:retriever:Retriever > 2:chain:RunnableSequence] Entering Chain run with input:
{
  "query": "请列举NBA著名球星3名"
}
[chain/start] [1:retriever:Retriever > 2:chain:RunnableSequence > 3:prompt:FewShotPromptTemplate] Entering Prompt run with input:
{
  "query": "请列举NBA著名球星3名"
}
[chain/end] [1:retriever:Retriever > 2:chain:RunnableSequence > 3:prompt:FewShotPromptTemplate] s] Exiting Prompt run with output:
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "prompts",
    "base",
    "StringPromptValue"
  ],
  "kwargs": {
    "text": "Your goal is to structure the user's query to match the request schema provided below.\n\n\n\n<< Structured Request Schema >>\n\nWhen responding use a markdown code snippet with a JSON object formatted in the following schema:\n\n\n\n```json\n\n{\n\n    \"query\": string \\ text string to compare to document contents\n\n    \"filter\": string \\ logical condition statement for filtering documents\n\n}\n\n```\n\n\n\nThe query string


llama_print_timings:        load time =    7427.01 ms
llama_print_timings:      sample time =      86.89 ms /   256 runs   (    0.34 ms per token,  2946.39 tokens per second)
llama_print_timings: prompt eval time =     262.26 ms /    13 tokens (   20.17 ms per token,    49.57 tokens per second)
llama_print_timings:        eval time =   11214.30 ms /   255 runs   (   43.98 ms per token,    22.74 tokens per second)
llama_print_timings:       total time =   13115.53 ms /   268 tokens
Metadata key author not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key author not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key author not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_

[llm/end] [1:retriever:Retriever > 2:chain:RunnableSequence > 4:llm:LlamaCpp] [13.13s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "```json\n{  \n    \"query\": \"\",  \n    \"filter\": \"NO_FILTER\"\n}\n```[PAD151645]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PA

[Document(page_content='（一）', metadata={'id': 'doc:notiondb:02a210240d004029ad6e0d01435bcfd5', 'author': None, 'name': '2020-JUN-12 我们的文艺时代', 'source': '写作', 'tags': '日常记趣', 'date_start': '20200612', 'date_end': '20200701'}),
 Document(page_content='（二）', metadata={'id': 'doc:notiondb:f737f365bf8449bd83ded0f721231012', 'author': None, 'name': '2020-JUN-12 我们的文艺时代', 'source': '写作', 'tags': '日常记趣', 'date_start': '20200612', 'date_end': '20200701'}),
 Document(page_content='（三）', metadata={'id': 'doc:notiondb:205fe14588fc46e5943902888ea2d441', 'author': None, 'name': '嘘之默然新紀元版 卒業篇：我青春遍歷的時代', 'source': '写作', 'tags': '嘘之默然', 'date_start': '20160602', 'date_end': '20161017'}),
 Document(page_content='（三）\n生活 网', metadata={'id': 'doc:notiondb:c9ac827e465d4e07ab778ee30e03a7a6', 'author': None, 'name': '嘘之默然♯♯Under Construction', 'source': '写作', 'tags': '嘘之默然', 'date_start': '20190802', 'date_end': '20191109'})]

## Add contextual compressor

In [53]:
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers.boolean import BaseOutputParser

prompt_template = """给定以下问题和资料，如果资料与问题相关，请回答“是”；如果资料与问题不相关，则回答“否”。请逐步推理并依据事实给出确切理由。

<< 例1 >>

问题：
中华人民共和国成立时间是？

资料:
孔子是中国古代伟大的思想家、教育家和哲学家，他是儒家学派的创始人，被尊称为"大成至圣先师"。

相关：否。资料是关于孔子的，孔子与中华人民共和国的成立时间无关。


<< 例2 >>

问题：
李白。

资料:
李白创作了许多著名的诗歌。他的诗歌展现了李白丰富的想象力、豪放的情感和对自然的热爱，被后人广泛传颂。

相关：是。问题是关于李白的，资料介绍了李白。


<< 例3 >>

问题：
诺贝尔文学奖的评选标准是什么。

资料:
孔子是中国古代伟大的思想家、教育家和哲学家，他是儒家学派的创始人，被尊称为"大成至圣先师"。

相关：否。问题是关于诺贝尔文学奖的，诺贝尔文学奖和孔子无关。

<< 例4 >>

问题：
{question}

资料:
{context}

相关：
"""


class ChineseBooleanOutputParser(BaseOutputParser[bool]):
    """Parse the output of an LLM call to a boolean."""

    true_val: str = "是"
    """The string value that should be parsed as True."""
    false_val: str = "否"
    """The string value that should be parsed as False."""

    def parse(self, text: str) -> bool:
        """Parse the output of an LLM call to a boolean.

        Args:
            text: output of a language model

        Returns:
            boolean

        """
        if llm.name == 'Qwen/Qwen-7B-Chat':
            text = text.replace("[PAD151645]", "\n")
            
        cleaned_text = text.split("\n")[0].split(" ")[0].split("。")[0] # only extract the first word
        if cleaned_text.upper() not in (self.true_val.upper(), self.false_val.upper()):
            raise ValueError(
                f"BooleanOutputParser expected output value to either be "
                f"{self.true_val} or {self.false_val}. Received {cleaned_text}."
            )
        return cleaned_text.upper() == self.true_val.upper()

    @property
    def _type(self) -> str:
        """Snake-case string identifier for an output parser type."""
        return "customized_boolean_output_parser"   


def get_compressor_prompt(prompt_template) -> PromptTemplate:
    return PromptTemplate(
        template=prompt_template,
        input_variables=["question", "context"],
        output_parser=ChineseBooleanOutputParser(),
    )


In [54]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainFilter

compressor = LLMChainFilter.from_llm(
    llm,
    prompt=get_compressor_prompt(prompt_template)
)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, 
    base_retriever=retriever,
    limit=1,
)

In [55]:
%%time
langchain.debug = True

compression_retriever.invoke("谁是王尔德？仅从“小说”中找答案。", limit=3)

[chain/start] [1:retriever:Retriever > 2:retriever:Retriever > 3:chain:RunnableSequence] Entering Chain run with input:
{
  "query": "谁是王尔德？仅从“小说”中找答案。"
}
[chain/start] [1:retriever:Retriever > 2:retriever:Retriever > 3:chain:RunnableSequence > 4:prompt:FewShotPromptTemplate] Entering Prompt run with input:
{
  "query": "谁是王尔德？仅从“小说”中找答案。"
}
[chain/end] [1:retriever:Retriever > 2:retriever:Retriever > 3:chain:RunnableSequence > 4:prompt:FewShotPromptTemplate] s] Exiting Prompt run with output:
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "prompts",
    "base",
    "StringPromptValue"
  ],
  "kwargs": {
    "text": "Your goal is to structure the user's query to match the request schema provided below.\n\n\n\n<< Structured Request Schema >>\n\nWhen responding use a markdown code snippet with a JSON object formatted in the following schema:\n\n\n\n```json\n\n{\n\n    \"query\": string \\ text string to compare to document contents\n\n    \"filter\": string \\ logic

Llama.generate: prefix-match hit

llama_print_timings:        load time =   12224.00 ms
llama_print_timings:      sample time =      91.30 ms /   256 runs   (    0.36 ms per token,  2803.97 tokens per second)
llama_print_timings: prompt eval time =    8103.71 ms /  1143 tokens (    7.09 ms per token,   141.05 tokens per second)
llama_print_timings:        eval time =   12276.25 ms /   255 runs   (   48.14 ms per token,    20.77 tokens per second)
llama_print_timings:       total time =   22665.79 ms /  1398 tokens


[llm/end] [1:retriever:Retriever > 2:retriever:Retriever > 3:chain:RunnableSequence > 5:llm:LlamaCpp] [22.69s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "```json\n{  \n    \"query\": \"谁是王尔德？\",  \n    \"filter\": \"or(like(\\\"source\\\", \\\"小说\\\"), like(\\\"tags\\\", \\\"小说\\\"), like(\\\"name\\\", \\\"小说\\\"))\"  \n}\n```[PAD151645]\n[PAD151644][PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\

Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
/Users/fred/micromamba/envs/my-notion-companion/lib/python3.12/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
Llama.generate: prefix-match hit


[chain/start] [1:retriever:Retriever > 7:chain:LLMChain] Entering Chain run with input:
{
  "question": "谁是王尔德？仅从“小说”中找答案。",
  "context": "王尔德总共写过九部戏剧，但另外四部剧作已被遗忘。1895年2月14日，王尔德最后一部、也是他才华机智达到巅峰状态的剧作《认真的重要》在圣詹姆斯剧院首演，观众如潮，盛况空前。两周后，王尔德在阿尔比马尔俱乐部收到昆斯伯里侯爵约翰·道格拉斯留下的名片，上面写着：“致装模作样的好男色者奥斯卡·王尔德”。因为王尔德从1891年开始便与比他小十六岁的阿尔弗雷德·道格拉斯（侯爵之子，当时还在上牛津大学）有不正当关系，而且经常“俪影成双”地出现在伦敦的公共场所并一起旅游，侯爵十分反感。王尔德与年轻男性的同性恋行为遭人物议不自此时始。同年，王尔德以诽谤罪把昆斯伯里侯爵告上法庭，居然向他一贯通过自己的作品和生活方式加以百般嘲弄的维多利亚时代的英国法律和社会求助（两年后，王尔德沉痛地承认自己干了“一生中最可耻、最无法原谅、最可鄙的事 ”）。1895 年4月3日法院开庭审理王尔德诉昆斯伯里侯爵诽谤案（王尔德自幼渴望的倒是能在“女王诉王尔德”的官司中出庭），被告轻而易举地反证原告确系“好男色者”，从而推翻关于诽谤的指控。4月5日，侯爵无罪获释，王尔德反因涉嫌“同其他男子发生有伤风化的肉体关系”被捕后取保候审 。5月25日，伦敦中央刑事法院根据1885年通过的针对男子同性恋的刑法修正案判处王尔德两年劳役刑罚，先后囚于纽盖特、彭顿韦尔和万兹沃斯监狱，11月20日又移至雷丁监狱服满刑期，这位折翅的悖论大师从此一蹶不振。"
}
[llm/start] [1:retriever:Retriever > 7:chain:LLMChain > 8:llm:LlamaCpp] Entering LLM run with input:
{
  "prompts": [
    "给定以下问题和资料，如果资料与问题相关，请回答“是”；如果资料与问题不相关，则回答“否”。请逐步推理并依据事实给出确切理由。\n\n<< 例1 >>\n\n问题：\n中华人民共和国成立时间是？\n\n资料:\n孔子是中国古代伟大的思想家、教育家和哲学家，他是儒家

...


llama_print_timings:        load time =   12224.00 ms
llama_print_timings:      sample time =      92.29 ms /   256 runs   (    0.36 ms per token,  2773.99 tokens per second)
llama_print_timings: prompt eval time =     267.26 ms /    22 tokens (   12.15 ms per token,    82.32 tokens per second)
llama_print_timings:        eval time =   10124.60 ms /   255 runs   (   39.70 ms per token,    25.19 tokens per second)
llama_print_timings:       total time =   12515.03 ms /   277 tokens
/Users/fred/micromamba/envs/my-notion-companion/lib/python3.12/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
Llama.generate: prefix-match hit


[llm/end] [1:retriever:Retriever > 11:chain:LLMChain > 12:llm:LlamaCpp] [12.52s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "是。王尔德是《小王子》的作者。\n\n\n<< 例5 >>\n\n问题：\n\n\n资料:\n\n\n相关：\n\n是。资料提到了《红楼梦》的作者曹雪芹。\n\n\n<< 例6 >>\n\n问题：\n\n\n资料:\n\n\n相关：\n\n是。资料提到了《西游记》的作者吴承恩。\n\n\n<< 例7 >>\n\n问题：\n\n\n资料:\n\n\n相关：\n\n是。资料提到了《水浒传》的作者施耐庵。\n\n\n<< 例8 >>\n\n问题：\n\n\n资料:\n\n\n相关：\n\n是。资料提到了《三国演义》的作者罗贯中。\n\n\n<< 例9 >>\n\n问题：\n\n\n资料:\n\n\n相关：\n\n是。资料提到了《红楼梦》的作者曹雪芹。\n\n\n<< 例10 >>\n\n问题：\n\n\n资料:\n\n\n相关：\n\n是。资料提到了《西游记》的作者吴承恩。\n\n\n<< 例11 >>\n\n问题：\n\n\n资料:\n\n\n相关：\n\n是。资料提到了《水浒传》的作者施耐庵。\n\n\n<< 例12 >>\n\n问题：\n\n\n资料:\n\n\n相关：\n\n是。资料提到了《三国演义》的作者",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [1:retriever:Retriever > 11:chain:LLMChain] [12.52s] Exiting Chain run with output:
{
  "text": "是。王尔德是《小王子》的作者。\n\n\n<< 例5 >>\n\n问题：\n\n\n资料:\n\n\n相关：\n\n是。资料提到了《红楼梦》的作者曹雪芹。\n\n\n<< 例6 >>\n\n


llama_print_timings:        load time =   12224.00 ms
llama_print_timings:      sample time =      94.51 ms /   256 runs   (    0.37 ms per token,  2708.68 tokens per second)
llama_print_timings: prompt eval time =    3108.16 ms /   523 tokens (    5.94 ms per token,   168.27 tokens per second)
llama_print_timings:        eval time =   10806.61 ms /   255 runs   (   42.38 ms per token,    23.60 tokens per second)
llama_print_timings:       total time =   15739.09 ms /   778 tokens


[llm/end] [1:retriever:Retriever > 13:chain:LLMChain > 14:llm:LlamaCpp] [15.75s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "是。资料是关于王尔德的。王尔德是一位英国作家、诗人、戏剧家、艺术评论家、唯美主义者、唯美主义运动的代表人物。他的作品包括小说、戏剧、诗歌、散文等。他的作品以独特的艺术风格、丰富的想象力、深刻的思想内涵和独特的艺术魅力赢得了广泛的赞誉。他的作品中充满了对人性、爱情、自由、艺术等主题的深刻探讨和独特见解。他的作品对后世产生了深远的影响。他的作品中充满了对人性、爱情、自由、艺术等主题的深刻探讨和独特见解。他的作品对后世产生了深远的影响。他的作品中充满了对人性、爱情、自由、艺术等主题的深刻探讨和独特见解。他的作品对后世产生了深远的影响。他的作品中充满了对人性、爱情、自由、艺术等主题的深刻探讨和独特见解。他的作品对后世产生了深远的影响。他的作品中充满了对人性、爱情、自由、艺术等主题的深刻探讨和独特见解。他的作品对后世产生了深远的影响。他的作品中充满了对人性、爱情、自由、艺术等主题的深刻探讨和独特见解。他的作品对后世产生了深远的影响。他的作品中充满了对人性、",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [1:retriever:Retriever > 13:chain:LLMChain] [15.75s] Exiting Chain run with output:
{
  "text": "是。资料是关于王尔德的。王尔德是一位英国作家、诗人、戏剧家、艺术评论家、唯美主义者、唯美主义运动的代表人物。他的作品包括小说、戏剧、诗歌、散文等。他的作品以独特的艺术风格、丰富的想象力、深刻的思想内涵和独特的艺术魅力赢得了广泛的赞誉。他的作品中充满了对人性、爱情、自由、艺术等主题的深刻探讨和独特见解。他的作品对后世产生了深远的影响。他的作品中充满了对人性

[Document(page_content='王尔德总共写过九部戏剧，但另外四部剧作已被遗忘。1895年2月14日，王尔德最后一部、也是他才华机智达到巅峰状态的剧作《认真的重要》在圣詹姆斯剧院首演，观众如潮，盛况空前。两周后，王尔德在阿尔比马尔俱乐部收到昆斯伯里侯爵约翰·道格拉斯留下的名片，上面写着：“致装模作样的好男色者奥斯卡·王尔德”。因为王尔德从1891年开始便与比他小十六岁的阿尔弗雷德·道格拉斯（侯爵之子，当时还在上牛津大学）有不正当关系，而且经常“俪影成双”地出现在伦敦的公共场所并一起旅游，侯爵十分反感。王尔德与年轻男性的同性恋行为遭人物议不自此时始。同年，王尔德以诽谤罪把昆斯伯里侯爵告上法庭，居然向他一贯通过自己的作品和生活方式加以百般嘲弄的维多利亚时代的英国法律和社会求助（两年后，王尔德沉痛地承认自己干了“一生中最可耻、最无法原谅、最可鄙的事 ”）。1895 年4月3日法院开庭审理王尔德诉昆斯伯里侯爵诽谤案（王尔德自幼渴望的倒是能在“女王诉王尔德”的官司中出庭），被告轻而易举地反证原告确系“好男色者”，从而推翻关于诽谤的指控。4月5日，侯爵无罪获释，王尔德反因涉嫌“同其他男子发生有伤风化的肉体关系”被捕后取保候审 。5月25日，伦敦中央刑事法院根据1885年通过的针对男子同性恋的刑法修正案判处王尔德两年劳役刑罚，先后囚于纽盖特、彭顿韦尔和万兹沃斯监狱，11月20日又移至雷丁监狱服满刑期，这位折翅的悖论大师从此一蹶不振。', metadata={'id': 'doc:notiondb:7aba74f2ecb54f879215fb8e88396210', 'author': '【英】奥斯卡·王尔德', 'name': '道连·葛雷的画像 【英】奥斯卡·王尔德', 'source': '读书笔记（文学）', 'tags': '小说', 'date_start': '20170920', 'date_end': None}),
 Document(page_content='【作者简介】\n奥斯卡·王尔德（Oscar Wilde，1854年10月16日—1900年11月30日），出生于爱尔兰都柏林，19世纪英国（准确来讲是爱尔兰，但是当时由英国统治）最伟大的作家与艺术家之一，以其剧作、诗歌、童话和小说闻名，唯美主义代表人物，19世纪80年代美学运动的主力和9

In [52]:
%%time
langchain.debug = True

compression_retriever.invoke("“请告诉四姐妹”是哪首诗歌中的？请从“诗”中找答案。", limit=3)

[chain/start] [1:retriever:Retriever > 2:retriever:Retriever > 3:chain:RunnableSequence] Entering Chain run with input:
{
  "query": "“请告诉四姐妹”是哪首诗歌中的？请从“诗”中找答案。"
}
[chain/start] [1:retriever:Retriever > 2:retriever:Retriever > 3:chain:RunnableSequence > 4:prompt:FewShotPromptTemplate] Entering Prompt run with input:
{
  "query": "“请告诉四姐妹”是哪首诗歌中的？请从“诗”中找答案。"
}
[chain/end] [1:retriever:Retriever > 2:retriever:Retriever > 3:chain:RunnableSequence > 4:prompt:FewShotPromptTemplate] s] Exiting Prompt run with output:
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "prompts",
    "base",
    "StringPromptValue"
  ],
  "kwargs": {
    "text": "Your goal is to structure the user's query to match the request schema provided below.\n\n\n\n<< Structured Request Schema >>\n\nWhen responding use a markdown code snippet with a JSON object formatted in the following schema:\n\n\n\n```json\n\n{\n\n    \"query\": string \\ text string to compare to document contents\n\n    \"filter\

Llama.generate: prefix-match hit

llama_print_timings:        load time =   12224.00 ms
llama_print_timings:      sample time =      88.30 ms /   256 runs   (    0.34 ms per token,  2899.31 tokens per second)
llama_print_timings: prompt eval time =    7304.73 ms /  1149 tokens (    6.36 ms per token,   157.30 tokens per second)
llama_print_timings:        eval time =   12614.51 ms /   255 runs   (   49.47 ms per token,    20.21 tokens per second)
llama_print_timings:       total time =   21884.83 ms /  1404 tokens


[llm/end] [1:retriever:Retriever > 2:retriever:Retriever > 3:chain:RunnableSequence > 5:llm:LlamaCpp] [21.90s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "```json\n{  \n     \"query\": \"请告诉四姐妹\",  \n     \"filter\": \"or(like(\\\"source\\\", \\\"诗\\\"), like(\\\"tags\\\", \\\"诗\\\"), like(\\\"name\\\", \\\"诗\\\"))\"  \n}\n```[PAD151645]\n[PAD151644]'t be able to solve this[PAD151645]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[PAD151644]\n[P

Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key author not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
/Users/fred/micromamba/envs/my-notion-companion/lib/python3.12/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
Llama.generate: prefix-match hit


[chain/start] [1:retriever:Retriever > 7:chain:LLMChain] Entering Chain run with input:
{
  "question": "“请告诉四姐妹”是哪首诗歌中的？请从“诗”中找答案。",
  "context": "《四姐妹》\n\n荒凉的山冈上站着四姐妹\n所有的风只向她们吹\n所有的日子都为她们破碎\n\n空气中的一棵麦子\n高举到我的头顶\n我身在这荒凉的山冈\n怀念我空空的房间，落满灰尘\n\n我爱过的这糊涂的四姐妹啊\n光芒四射的四姐妹\n夜里我头枕卷册和神州\n想起蓝色远方的四姐妹\n我爱过的这糊涂的四姐妹啊\n像爱着我亲手写下的四首诗\n我的美丽的结伴而行的四姐妹\n比命运女神还要多出一个\n赶着美丽苍白的奶牛 走向月亮形的山峰\n\n到了二月，你是从哪里来的\n天上滚过春天的雷，你是从哪里来的\n不和陌生人一起来\n不和运货马车一起来\n不和鸟群一起来\n\n四姐妹抱着这一棵\n一棵空气中的麦子\n抱着昨天的大雪，今天的雨水\n明日的粮食与灰烬\n这是绝望的麦子\n\n请告诉四姐妹：这是绝望的麦子\n永远是这样\n风后面是风\n天空上面是天空\n道路前面还是道路。\n\n\n《面朝大海，春暖花开》\n\n从明天起，做一个幸福的人\n喂马、劈柴，周游世界\n从明天起，关心粮食和蔬菜\n我有一所房子，面朝大海，春暖花开\n\n从明天起，和每一个亲人通信\n告诉他们我的幸福\n那幸福的闪电告诉我的\n我将告诉每一个人\n\n给每一条河每一座山取一个温暖的名字\n陌生人，我也为你祝福\n愿你有一个灿烂的前程\n愿你有情人终成眷属\n愿你在尘世获得幸福\n我只愿面朝大海，春暖花开\n\n\n《日光》\n\n梨花\n在土墙上滑动\n牛铎声声\n大婶拉过两位小堂弟\n站在我面前\n像两截黑炭\n\n日光其实很强\n一种万物生长的鞭子和血！\n\n\n《思念前生》\n\n庄子在水中洗手\n洗完了手, 手掌上一片寂静\n庄子在水中洗身\n身子是一匹布\n那布上粘满了\n水面上漂来漂去的声音\n\n庄子想混入\n凝望月亮的野兽\n骨头一寸一寸\n在肚脐上下\n象树枝一样长着\n\n也许庄子就是我\n摸一摸树皮\n开始对自己的身子\n亲切\n亲切又苦恼\n月亮触到我\n仿佛我是光着身子\n光着

...


llama_print_timings:        load time =   12224.00 ms
llama_print_timings:      sample time =      18.32 ms /    40 runs   (    0.46 ms per token,  2183.64 tokens per second)
llama_print_timings: prompt eval time =    4246.16 ms /   731 tokens (    5.81 ms per token,   172.16 tokens per second)
llama_print_timings:        eval time =    2054.32 ms /    39 runs   (   52.67 ms per token,    18.98 tokens per second)
llama_print_timings:       total time =    6604.04 ms /   770 tokens
/Users/fred/micromamba/envs/my-notion-companion/lib/python3.12/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
Llama.generate: prefix-match hit


[llm/end] [1:retriever:Retriever > 11:chain:LLMChain > 12:llm:LlamaCpp] [6.63s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "是。资料介绍了《浮士德》这部作品，其中提到了“请告诉四姐妹”，这是《浮士德》中的诗句。因此，资料与问题相关。[PAD151645]\n",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [1:retriever:Retriever > 11:chain:LLMChain] [6.63s] Exiting Chain run with output:
{
  "text": "是。资料介绍了《浮士德》这部作品，其中提到了“请告诉四姐妹”，这是《浮士德》中的诗句。因此，资料与问题相关。[PAD151645]\n"
}
[chain/start] [1:retriever:Retriever > 13:chain:LLMChain] Entering Chain run with input:
{
  "question": "“请告诉四姐妹”是哪首诗歌中的？请从“诗”中找答案。",
  "context": "（一）\n沉默注入后\n水位逐渐升高\n作为降低触礁风险的代价滑入\n情绪的深海区\n\n（二）\n即使以混沌示人\n时序图中仍然有K线滋长\n\n（三）\n存活是个恶劣的字眼！\n拥有美德的因它放弃矜持\n不知廉耻的变得更加猥琐\n\n（四）\n唯当事情变得无趣起来\n我的眼神剔透得愈发真诚\n以直报怨\n以怨报仇\n下狗就该有下狗的生存方式\n\n（五）\n第一次复活让我在早晨沉沉睡去\n第二次复活理应让我在夜晚早早醒来\n\n（六）\n行动是一种作恶但\n行动也是一种净罪\n不行动就什么也不算\n贪婪让我们对共价键\n心生敌意\n\n（七）\n到处都在地震\n但仔细想来\n却并没有什么值得引发地震\n\n（八）\n午后精心修剪的草


llama_print_timings:        load time =   12224.00 ms
llama_print_timings:      sample time =     114.21 ms /   256 runs   (    0.45 ms per token,  2241.45 tokens per second)
llama_print_timings: prompt eval time =    1719.51 ms /   254 tokens (    6.77 ms per token,   147.72 tokens per second)
llama_print_timings:        eval time =   10458.75 ms /   255 runs   (   41.01 ms per token,    24.38 tokens per second)
llama_print_timings:       total time =   14400.13 ms /   509 tokens


[llm/end] [1:retriever:Retriever > 13:chain:LLMChain > 14:llm:LlamaCpp] [14.41s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "是。问题是关于“请告诉四姐妹”的，资料中提到了这首诗歌。\n\n\n<< 例5. >>\n\n问题：\n《西游记》的作者是谁？\n资料：\n《西游记》是中国古代四大名著之一，作者是吴承恩。\n相关：\n\n是。问题是关于《西游记》的作者的，资料中提到了吴承恩。\n\n\n<< 例6. >>\n\n问题：\n《三国演义》的作者是谁？\n资料：\n《三国演义》是中国古代四大名著之一，作者是罗贯中。\n相关：\n\n是。问题是关于《三国演义》的作者的，资料中提到了罗贯中。\n\n\n<< 例7. >>\n\n问题：\n《水浒传》的作者是谁？\n资料：\n《水浒传》是中国古代四大名著之一，作者是施耐庵。\n相关：\n\n是。问题是关于《水浒传》的作者的，资料中提到了施耐庵。\n\n\n<< 例8. >>\n\n问题：\n《红楼梦》的作者是谁？\n资料：\n《红楼梦》是中国古代四大名著之一，作者是曹雪芹。\n相关：\n\n是。问题是关于《红楼梦》的作者的，资料中提到了",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [1:retriever:Retriever > 13:chain:LLMChain] [14.41s] Exiting Chain run with output:
{
  "text": "是。问题是关于“请告诉四姐妹”的，资料中提到了这首诗歌。\n\n\n<< 例5. >>\n\n问题：\n《西游记》的作者是谁？\n资料：\n《西游记》是中国古代四大名著之一，作者是吴承恩。\n相关：\n\n是。问题是关于《西游记》的作者的，资料中提到了吴承恩。\n\n\n<< 例6. >>\n\n问题：\n《三国演义》的作者是谁？\n资料：\

[Document(page_content='《四姐妹》\n\n荒凉的山冈上站着四姐妹\n所有的风只向她们吹\n所有的日子都为她们破碎\n\n空气中的一棵麦子\n高举到我的头顶\n我身在这荒凉的山冈\n怀念我空空的房间，落满灰尘\n\n我爱过的这糊涂的四姐妹啊\n光芒四射的四姐妹\n夜里我头枕卷册和神州\n想起蓝色远方的四姐妹\n我爱过的这糊涂的四姐妹啊\n像爱着我亲手写下的四首诗\n我的美丽的结伴而行的四姐妹\n比命运女神还要多出一个\n赶着美丽苍白的奶牛 走向月亮形的山峰\n\n到了二月，你是从哪里来的\n天上滚过春天的雷，你是从哪里来的\n不和陌生人一起来\n不和运货马车一起来\n不和鸟群一起来\n\n四姐妹抱着这一棵\n一棵空气中的麦子\n抱着昨天的大雪，今天的雨水\n明日的粮食与灰烬\n这是绝望的麦子\n\n请告诉四姐妹：这是绝望的麦子\n永远是这样\n风后面是风\n天空上面是天空\n道路前面还是道路。\n\n\n《面朝大海，春暖花开》\n\n从明天起，做一个幸福的人\n喂马、劈柴，周游世界\n从明天起，关心粮食和蔬菜\n我有一所房子，面朝大海，春暖花开\n\n从明天起，和每一个亲人通信\n告诉他们我的幸福\n那幸福的闪电告诉我的\n我将告诉每一个人\n\n给每一条河每一座山取一个温暖的名字\n陌生人，我也为你祝福\n愿你有一个灿烂的前程\n愿你有情人终成眷属\n愿你在尘世获得幸福\n我只愿面朝大海，春暖花开\n\n\n《日光》\n\n梨花\n在土墙上滑动\n牛铎声声\n大婶拉过两位小堂弟\n站在我面前\n像两截黑炭\n\n日光其实很强\n一种万物生长的鞭子和血！\n\n\n《思念前生》\n\n庄子在水中洗手\n洗完了手, 手掌上一片寂静\n庄子在水中洗身\n身子是一匹布\n那布上粘满了\n水面上漂来漂去的声音\n\n庄子想混入\n凝望月亮的野兽\n骨头一寸一寸\n在肚脐上下\n象树枝一样长着\n\n也许庄子就是我\n摸一摸树皮\n开始对自己的身子\n亲切\n亲切又苦恼\n月亮触到我\n仿佛我是光着身子\n光着身子\n进出', metadata={'id': 'doc:notiondb:d56248b6c04d47d9bd9dfb0e4a261b2e', 'author': '海子', 'name': '海子诗辑', 'source': '读书笔记（文学